# Non-Planar Sine Wave Sample Generator
  
press ctrl+F9 to run all cells in this notebook, or press shift+enter to run each cell sequentially

if you change one of the code cells, make sure you run it and all subsequent cells again (in order)

*this document is a jupyter notebook - if they're new to you, check out how they work: [link](https://www.google.com/search?q=ipynb+tutorial), [link](https://jupyter.org/try-jupyter/retro/notebooks/?path=notebooks/Intro.ipynb), [link](https://colab.research.google.com/)*
### be patient :)

the next code cell may take a while because running it causes several things to happen:
- connect to a google colab server -> download the fullcontrol code -> install the fullcontrol code

check out [other tutorials](https://github.com/FullControlXYZ/fullcontrol/blob/master/docs/README.md) to understand the python code for the FullControl design

In [1]:
!pip install git+https://github.com/FullControlXYZ/fullcontrol --quiet
import fullcontrol as fc
import lab.fullcontrol as fclab
from math import tau, radians
import numpy as np

In [2]:
# printer/gcode parameters

design_name = 'Sine Bone Sample Compression'
nozzle_temp = 200
bed_temp = 60
print_speed = 750
fan_percent = 100
printer_name='generic' # generic / ultimaker2plus / prusa_i3 / ender_3 / cr_10 / bambulab_x1 / toolchanger_T0

In [3]:
# design parameters

EW = 0.6 # extrusion width
EH = 0.2 # extrusion height (and layer height)
initial_z = EH*0.6 # initial nozzle position is set to 0.6x the extrusion height to get a bit of 'squish' for good bed adhesion

solid_layers = 0 #number of solid layers at the top/bottom to mount the sample
mount_layers = 20 #number of layers at the top/bottom to transition from solid to 
sweeping_layers = 0 #number of layers between the thick and thin sections
sample_layers = 85 #number of layers in the thin section of the sample


sampleWidth = 8 # width of the sample probs gonna make this an array to vary width with height
amplitude = 0 #width/2 added to the section that is clamped
squareSize = sampleWidth+2*amplitude
sine_height_max = 0.6 #3* layer height

brimCount = 16
resolution = 15 #odd to ensure peak is within the samples

originX = 90 #center X of the design
originY = 90 #center Y of the design

In [4]:
# generate the design (make sure you've run the above cells before running this cell)
sine_height = 0
steps = []
width = sampleWidth + 2*amplitude
ZZfade = 0

radius = width/2 #I cant think of a better variable name soz
sampleR = sampleWidth/2 #same^

def x_layer():
    
    
    radius = width/2 #I cant think of a better variable name soz
    sampleR = sampleWidth/2 #same^
    wave_Z = np.linspace(0, 2*np.pi, resolution)
    wave_Z = sine_height * (-np.cos(wave_Z)+1)/2
    xy_array = np.linspace(0, radius, resolution)

    
    #square
    steps.extend(fc.rectangleXY(fc.Point(x=originX-radius, y=originY-radius, z=initial_z+layer*EH), width, width))
    
    #cross
    for point in range(0,resolution):
        steps.append(fc.Point(x=originX-np.flip(xy_array)[point], y=originY-np.flip(xy_array)[point], z=initial_z+layer*EH+wave_Z[point]))
    for point in range(resolution):
        steps.append(fc.Point(x=originX+xy_array[point], y=originY+xy_array[point], z=initial_z+layer*EH+wave_Z[point]))
        
    steps.extend([fc.Extruder(on=False), fc.Point(x=originX-radius, y=originY+radius, z=initial_z+layer*EH), fc.Extruder(on=True)])
    
    for point in range(resolution):
        steps.append(fc.Point(x=originX-np.flip(xy_array)[point], y=originY+np.flip(xy_array)[point], z=initial_z+layer*EH+wave_Z[point]))
    for point in range(resolution):
        steps.append(fc.Point(x=originX+xy_array[point], y=originY-xy_array[point], z=initial_z+layer*EH+wave_Z[point]))
        #print(np.flip(xy_array)[point])
    
        
    #diamond
    steps.extend([fc.Extruder(on=False), fc.Point(x=originX, y=originY-radius, z=initial_z+layer*EH), fc.Extruder(on=True)])
    for point in range(resolution):
        steps.append(fc.Point(x=originX+xy_array[point], y=originY-np.flip(xy_array)[point], z=initial_z+layer*EH+wave_Z[point]))
        
    for point in range(resolution):
        steps.append(fc.Point(x=originX+np.flip(xy_array)[point], y=originY+xy_array[point], z=initial_z+layer*EH+wave_Z[point]))
        
    for point in range(resolution):
        steps.append(fc.Point(x=originX-xy_array[point], y=originY+np.flip(xy_array)[point], z=initial_z+layer*EH+wave_Z[point]))
        
    for point in range(resolution):
        steps.append(fc.Point(x=originX-np.flip(xy_array)[point], y=originY-xy_array[point], z=initial_z+layer*EH+wave_Z[point]))
        
    #Move to starting Position
    steps.extend([fc.Extruder(on=False), fc.Point(x=originX-radius, y=originY-radius, z=initial_z+layer*EH), fc.Extruder(on=True)])
    
    #steps.extend(fc.rectangleXY(fc.Point(x=originX-radius, y=originY-radius, z=initial_z+layer*EH), width, width))
    #moveto bottomright
    #steps.extend([fc.Extruder(on=False), fc.Point(x=originX+radius, y=originY-radius, z=initial_z+layer*EH), fc.Extruder(on=True)])
    #zigzag steps
    #steps.append(fc.Point(x=originX+sampleR, y=originY-sampleR, z=initial_z+layer*EH)) #flat under curve
    ##steps.append(fc.Point(x=originX+sampleR/2, y=originY-sampleR/2, z=initial_z+layer*EH+sine_height))
    #steps.append(fc.Point(x=originX, y=originY, z=initial_z+layer*EH))
    #steps.append(fc.Point(x=originX-sampleR/2, y=originY+sampleR/2, z=initial_z+layer*EH+sine_height))
    #steps.append(fc.Point(x=originX-sampleR, y=originY+sampleR, z=initial_z+layer*EH))
    #printto topleft
    #steps.append(fc.Point(x=originX-radius, y=originY+radius, z=initial_z+layer*EH))
    
    #moveto topright
    #steps.extend([fc.Extruder(on=False), fc.Point(x=originX+radius, y=originY+radius, z=initial_z+layer*EH), fc.Extruder(on=True)])
    #zigzag steps
    #steps.append(fc.Point(x=originX+sampleR, y=originY+sampleR, z=initial_z+layer*EH))
    #steps.append(fc.Point(x=originX+sampleR/2, y=originY+sampleR/2, z=initial_z+layer*EH+sine_height))
    #steps.append(fc.Point(x=originX, y=originY, z=initial_z+layer*EH))
    #steps.append(fc.Point(x=originX-sampleR/2, y=originY-sampleR/2, z=initial_z+layer*EH+sine_height))
    #steps.append(fc.Point(x=originX-sampleR, y=originY-sampleR, z=initial_z+layer*EH))
    #printto bottomleft
    #steps.append(fc.Point(x=originX-radius, y=originY-radius, z=initial_z+layer*EH))
    
    

#    if layer == 0:
#        if brimcount > 0:
#            steps.extend(fc.rectangleXY(fc.Point(x=originX-radius-brimcount*EW, y=originY-radius-brimcount*EW, z=initial_z), width+2*(brimcount*EW), width+2*(brimcount*EW)))
#            brimcounnt = brimcount-1
            
for layer in range (0,solid_layers):
    
    if layer<2:
        squareSize = squareSize+brimCount*EW
    
    steps.extend([fc.Extruder(on=False), fc.Point(x=originX-squareSize/2, y=originY-squareSize/2, z=initial_z+layer*EH), fc.Extruder(on=True)])
    while squareSize > 0.6:
        steps.extend(fc.rectangleXY(fc.Point(x=originX-squareSize/2, y=originY-squareSize/2, z=initial_z+layer*EH), squareSize, squareSize))
        squareSize = squareSize - 2*EW
    squareSize = width 
    
    
for layer in range (solid_layers,solid_layers+mount_layers):
    
    
    #bottom layers, thicker to allow the sample to be gripped
    x_layer()
    if sine_height < sine_height_max:
        sine_height = sine_height+0.06
        
    
for layer in range(solid_layers+mount_layers,solid_layers+mount_layers+sweeping_layers):

    #gradually decrease the width to the desired sample width
    width = sampleWidth + amplitude  + amplitude*np.cos((np.pi*(layer - (solid_layers+mount_layers))/(sweeping_layers)))
    
    x_layer()
    
for layer in range(solid_layers+mount_layers+sweeping_layers, solid_layers+mount_layers+sweeping_layers+sample_layers):

    #sample section that will fail
    x_layer()
    
for layer in range(solid_layers+mount_layers+sweeping_layers+sample_layers, solid_layers+mount_layers+sweeping_layers+sample_layers+sweeping_layers):

    #gradually increase the width back to the wider layers
    width = sampleWidth + amplitude  + amplitude*np.cos((np.pi + np.pi*(layer - (solid_layers+mount_layers+sweeping_layers+sample_layers))/sweeping_layers))
    
    x_layer()
    
for layer in range (solid_layers+mount_layers+sweeping_layers+sample_layers+sweeping_layers, solid_layers+mount_layers+sweeping_layers+sample_layers+sweeping_layers+mount_layers):
    
    #top layers, thicker to allow the sample to be gripped
    x_layer()
    if layer >= 2*(mount_layers+sweeping_layers)+sample_layers-12:
         if sine_height > 0:   
            sine_height = sine_height-0.06       
#move nozzle to prevent blob forming

for layer in range (solid_layers+mount_layers+sweeping_layers+sample_layers+sweeping_layers+mount_layers,solid_layers+mount_layers+sweeping_layers+sample_layers+sweeping_layers+mount_layers+solid_layers):
    if layer != (solid_layers+mount_layers+sweeping_layers+sample_layers+sweeping_layers+mount_layers):
        steps.extend([fc.Extruder(on=False), fc.Point(x=originX-width/2, y=originY-width/2, z=initial_z+layer*EH), fc.Extruder(on=True)])
    while squareSize > 0.6:
        steps.extend(fc.rectangleXY(fc.Point(x=originX-squareSize/2, y=originY-squareSize/2, z=initial_z+layer*EH), squareSize, squareSize))
        squareSize = squareSize - 2*EW
    squareSize = width 
    
steps.extend([fc.Extruder(on=False), fc.Point(x=originX-10, y=originY-10, z=initial_z+layer*EH+1), fc.Extruder(on=True)])
steps.append(fc.Point(x=originX-11, y=originY-10, z=initial_z+layer*EH+1)) #extrude a tiny amount to make the move happen           
    
# instead of the above for-loop code, you can create the exact same design using built-in FullControl functions (uncomment the next two lines):
# rectangle_steps = fc.rectangleXY(fc.Point(x=50, y=50, z=initial_z), 50, 50)
# steps = fc.move(rectangle_steps, fc.Vector(z=EH), copy=True, copy_quantity=layers)

In [5]:
# preview the design
fc.transform(steps, 'plot', fc.PlotControls(style='line', zoom=0.7))
# hover the cursor over the lines in the plot to check xyz positions of the points in the design

# uncomment the next line to create a plot with real heights/widths for extruded lines to preview the real 3D printed geometry
# fc.transform(steps, 'plot', fc.PlotControls(style='tube', zoom=0.7, initialization_data={'extrusion_width': EW, 'extrusion_height': EH}))

# uncomment the next line to create a neat preview (click the top-left button in the plot for a .png file) - post and tag @FullControlXYZ :)
# fc.transform(steps, 'plot', fc.PlotControls(neat_for_publishing=True, zoom=0.5, initialization_data={'extrusion_width': EW, 'extrusion_height': EH}))

In [6]:
# generate and save gcode

gcode_controls = fc.GcodeControls(
    printer_name=printer_name,

    initialization_data={
        'primer': 'front_lines_then_y',
        'print_speed': print_speed,
        'nozzle_temp': nozzle_temp,
        'bed_temp': bed_temp,
        'fan_percent': fan_percent,
        'extrusion_width': EW,
        'extrusion_height': EH})
gcode = fc.transform(steps, 'gcode', gcode_controls)
open(f'{design_name}.gcode', 'w').write(gcode)
#files.download(f'{design_name}.gcode')



574253

#### please tell us what you're doing with FullControl!

- tag FullControlXYZ on social media ([twitter](https://twitter.com/FullControlXYZ), [instagram](https://www.instagram.com/fullcontrolxyz/), [linkedin](https://www.linkedin.com/in/andrew-gleadall-068587119/), [tiktok](https://www.tiktok.com/@fullcontrolxyz))
- email [info@fullcontrol.xyz](mailto:info@fullcontrol.xyz)
- post on the [subreddit](https://reddit.com/r/fullcontrol)
- post in the [github discussions or issues tabs](https://github.com/FullControlXYZ/fullcontrol/issues)

in publications, please cite the original FullControl paper and the github repo for the new python version:

- Gleadall, A. (2021). FullControl GCode Designer: open-source software for unconstrained design in additive manufacturing. Additive Manufacturing, 46, 102109.
- Gleadall, A. and Leas, D. (2023). FullControl [electronic resource: python source code]. available at: https://github.com/FullControlXYZ/fullcontrol